<a href="https://colab.research.google.com/github/Oriolrt/MVC_M2_PGM/blob/main/1_EarthquakeExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Using the Earthquake network
In this example I will try to create the Alarm Bayesian Network using pgmpy and do some simple queries on the network. This network is mentioned in Bayesian Artificial Intelligence - Section 2.5.1 (https://bayesian-intelligence.com/publications/bai/book/BAI_Chapter2.pdf).
The actual model in this notebook and the CPD tables are the ones used in C2 of the Master of Computer Vision given by the Universitat Autònoma de Barcelona (UAB).

In this notebook we will play with the [PGM](https://pgmpy.org/) package. The full documentation is found in the [link](https://pgmpy.org/).

In [ ]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00


In [ ]:
# Importing Library
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.metrics.bn_inference import BayesianModelProbability
from pgmpy.inference import BeliefPropagation


The Bayesian network model is the one shown in the figure below:

<img src='https://drive.google.com/uc?id=1VHH4MV8d150FQcwbzL9Maq-cwa6KGTOM' height='400' >

The pgmpy code to built it is found in tyhe cell below:

In [ ]:
# Defining network structure

alarm_model = BayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "NeighborCalls"),
    ]
)

# Defining the parameters using CPT
from pgmpy.factors.discrete import TabularCPD

cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.7], [0.3]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.9], [0.1]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.99, 0.1, 0.3, 0.01], [0.01, 0.9, 0.7, 0.99]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_neighborcalls = TabularCPD(
    variable="NeighborCalls",
    variable_card=2,
    values=[[0.9, 0.2], [0.1, 0.8]],
    evidence=["Alarm"],
    evidence_card=[2],
    state_names={"NeighborCalls":["no","yes"],"Alarm":[False,True]},
)

# Associating the parameters with the model structure
alarm_model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_neighborcalls
)

In [ ]:
print(cpd_earthquake)
print(cpd_burglary)
print(cpd_alarm)
print(cpd_neighborcalls)



+---------------+-----+
| Earthquake(0) | 0.9 |
+---------------+-----+
| Earthquake(1) | 0.1 |
+---------------+-----+
+-------------+-----+
| Burglary(0) | 0.7 |
+-------------+-----+
| Burglary(1) | 0.3 |
+-------------+-----+
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.99          | 0.1           | 0.3           | 0.01          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.01          | 0.9           | 0.7           | 0.99          |
+------------+---------------+---------------+---------------+---------------+
+--------------------+--------------+-------------+
| Alarm

In [ ]:
# Checking if the cpds are valid for the model
alarm_model.check_model()



True

In [ ]:
# Viewing nodes of the model
alarm_model.nodes()



NodeView(('Burglary', 'Alarm', 'Earthquake', 'NeighborCalls'))

In [ ]:
# Viewing edges of the model
alarm_model.edges()



OutEdgeView([('Burglary', 'Alarm'), ('Alarm', 'NeighborCalls'), ('Earthquake', 'Alarm')])

In [ ]:
# Checking independcies of a node
alarm_model.local_independencies("Burglary")

(Burglary ⟂ Earthquake)

In [ ]:
# Listing all Independencies
alarm_model.get_independencies()



(NeighborCalls ⟂ Burglary, Earthquake | Alarm)
(NeighborCalls ⟂ Burglary | Alarm, Earthquake)
(NeighborCalls ⟂ Earthquake | Burglary, Alarm)
(Earthquake ⟂ Burglary)
(Earthquake ⟂ NeighborCalls | Alarm)
(Earthquake ⟂ NeighborCalls | Burglary, Alarm)
(Burglary ⟂ Earthquake)
(Burglary ⟂ NeighborCalls | Alarm)
(Burglary ⟂ NeighborCalls | Alarm, Earthquake)

In [ ]:
alarm_model.get_cpds()


[<TabularCPD representing P(Burglary:2) at 0x78a10a9c8670>,
 <TabularCPD representing P(Earthquake:2) at 0x78a1db9bc070>,
 <TabularCPD representing P(Alarm:2 | Burglary:2, Earthquake:2) at 0x78a1e8e73460>,
 <TabularCPD representing P(NeighborCalls:2 | Alarm:2) at 0x78a1e8e72d40>]

In [ ]:
from pgmpy.inference import VariableElimination

alarm_infer = VariableElimination(alarm_model)

In [ ]:
# Computing the joint probability of a Burglary and NeighborCalls.
q=alarm_infer.query(variables=["Burglary","NeighborCalls"])
print(q)

+-------------+--------------------+-------------------------------+
| Burglary    | NeighborCalls      |   phi(Burglary,NeighborCalls) |
+=============+====================+===============================+
| Burglary(0) | NeighborCalls(no)  |                        0.5815 |
+-------------+--------------------+-------------------------------+
| Burglary(0) | NeighborCalls(yes) |                        0.1185 |
+-------------+--------------------+-------------------------------+
| Burglary(1) | NeighborCalls(no)  |                        0.1169 |
+-------------+--------------------+-------------------------------+
| Burglary(1) | NeighborCalls(yes) |                        0.1831 |
+-------------+--------------------+-------------------------------+


In [ ]:
# Computing the probability of a Burglary given NeighborCalls="yes".
q=alarm_infer.query(variables=["Burglary"], evidence={"NeighborCalls": "yes"})
print(q)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.3929 |
+-------------+-----------------+
| Burglary(1) |          0.6071 |
+-------------+-----------------+


In [ ]:
# Computing the probability of a NeighborCalls  given Burglary=True.
q=alarm_infer.query(variables=["NeighborCalls"], evidence={"Burglary": True})
print(q)

+--------------------+----------------------+
| NeighborCalls      |   phi(NeighborCalls) |
+====================+======================+
| NeighborCalls(no)  |               0.3897 |
+--------------------+----------------------+
| NeighborCalls(yes) |               0.6103 |
+--------------------+----------------------+


In [ ]:
jt = alarm_model.to_junction_tree()
print(jt)
alarm_jt_infer = VariableElimination(jt)

q=alarm_jt_infer.query(variables=["Burglary"], evidence={"NeighborCalls": "yes"})
print(q)

JunctionTree with 2 nodes and 1 edges
+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.3929 |
+-------------+-----------------+
| Burglary(1) |          0.6071 |
+-------------+-----------------+
